# Explainable AI applied to assessors

In [2]:
from typing import *
from pathlib import Path

from transformers.models.auto.modeling_auto import AutoModelForSequenceClassification
from transformers.models.auto.tokenization_auto import AutoTokenizer
import numpy as np
import pandas as pd
import bigbench.api.results as bb

from lass.log_handling import LogLoader
from lass.datasets import split_instance_level, huggingfaceify

In [2]:
print("Loading data...")
loader = (LogLoader(logdir = Path('../artifacts/logs'))
        .with_tasks('paper-full')
        .with_model_families(['BIG-G T=0'])
        .with_model_sizes(['128b'])
        .with_shots([0])
        .with_query_types([bb.MultipleChoiceQuery])
)

_train, test = split_instance_level(loader, seed=42, test_fraction=0.2)
print("Data loaded.")

Loading data...


Data loaded.


In [3]:
import transformers
%env TOKENIZERS_PARALLELISM=true

dataset = huggingfaceify(_train[:1], test)
dataset['test'][0]

env: TOKENIZERS_PARALLELISM=true


{'text': 'In what follows, we provide short narratives, each of which illustrates a common proverb. \nNarrative: Barry was furious when the power went out-he wanted to read his new novel! He paced through the dark house, swearing angrily. But the more he ranted and raved, the worse he felt. Then Barry remembered he had a battery-powered lantern, so he dug it out and set it up. Pretty soon, one corner of the house was warmly lit and welcoming. Barry settled down happily with his book and waited for the power to return.\nThis narrative is a good illustration of the following proverb: ',
 'label': 1}

In [4]:
transformers.logging.set_verbosity_error() # type: ignore
model = AutoModelForSequenceClassification.from_pretrained("../artifacts/assessors/bert-bs32-0sh/checkpoint-1500")

# Tokenize according to specific model tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# This "transformer-interpret" library doesn't really deal well with truncation
# of long sequences, so we'll just truncate the sequences ourselves.
# 
# Note: Encoding is a destructive process,
# so we need to work with offset_mapping instead of just reversing.
# https://github.com/huggingface/tokenizers/issues/826#issuecomment-966082496
def truncate(batch):
   tokens = tokenizer(batch["text"], padding="max_length", truncation=True, 
      return_tensors="np", return_offsets_mapping=True)

   # We use `amax` as a trick to find the last non-zero offset mapping.
   # Array dimensions are [batch size, sequence length (in tokens), 2],
   # where the last dimension is [start, end] of the token (referring to index in the string).
   # With [:,:,-1], we make it [batch size, sequence length], taking only the end offset of the token.
   # Then we take the max of each sequence, producing batch_size numbers.
   lengths = np.amax(tokens['offset_mapping'][:,:,-1], axis=1) # type: ignore
   assert len(lengths) == len(batch['text'])

   # Now we cut all the strings
   texts = [text[:end] for text, end in zip(batch["text"], lengths)]
   return {'text': texts, 'label': batch['label']}

truncated_datasets = dataset.map(truncate, batched=True)
truncated_datasets['test'][0]

Parameter 'function'=<function truncate at 0x7f9fbbc571f0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

{'text': 'In what follows, we provide short narratives, each of which illustrates a common proverb. \nNarrative: Barry was furious when the power went out-he wanted to read his new novel! He paced through the dark house, swearing angrily. But the more he ranted and raved, the worse he felt. Then Barry remembered he had a battery-powered lantern, so he dug it out and set it up. Pretty soon, one corner of the house was warmly lit and welcoming. Barry settled down happily with his book and waited for the power to return.\nThis narrative is a good illustration of the following proverb:',
 'label': 1}

In [5]:
from transformers_interpret import SequenceClassificationExplainer
from collections import defaultdict

cls_explainer = SequenceClassificationExplainer(model, tokenizer) #type: ignore

# Print header
path = Path("xai.csv")
pd.DataFrame([],columns=['word', 'contribution', 'LM_score', 'Assr_pred']).to_csv(path, index=False)

# for index, instance in enumerate(truncated_datasets['test'].select(range(255, 260))):
for index, instance in enumerate(truncated_datasets['test']):
    if index % 50 == 0: # type: ignore
        print(f"{index}/{len(truncated_datasets['test'])}")

    # EXPLAINABILITY
    text, LM_correct = instance['text'], instance['label'] # type: ignore
    exp_neg = cls_explainer(text, class_name='LABEL_0')

    frame = pd.DataFrame(exp_neg, columns=['word', 'contribution'])
    frame['LM_score'] = LM_correct
    frame['Assr_pred'] = cls_explainer.predicted_class_index
    frame.to_csv(path, mode='a', header=False, index=False)

0/11004


50/11004


100/11004


150/11004


200/11004


...

10800/11004


10850/11004


10900/11004


10950/11004


11000/11004


In [17]:
words = pd.read_csv("xai_bert.csv")
mean = (words
    .query('LM_score == 1')
    .groupby('word')
    .agg( # type: ignore
        mean_contribution=('contribution', 'mean'),
        word_count=('word', 'count')
    )
    .sort_values('mean_contribution', ascending=False)
    .query('word_count > 10')
)
mean.to_csv("../artifacts/tmp/xai.csv")